In [ ]:
# Import Hail
import hail as hl

# Initialize Hail
hl.init()

# Import other things I may need
import pprint as pp
import pandas as pd
import numpy as np

In [ ]:
# Can we calculate a mutation rate per variant type per gene?

In [ ]:
# Let's look at mutation rate context table first
mut = hl.read_table('gs://my_bucket/Mutation_rates/gnomad.v4.1.mutation_rate.ht')

mut.count() # 104

In [ ]:
# Need to add reverse complements to be able to annotate all sequence contexts
mut.show(104)

In [ ]:
mut.describe()

In [ ]:
# So let's make reverse complement
rev = mut.key_by('methylation_level')
rev = rev.annotate(context = hl.reverse_complement(rev.context),
                   ref = hl.reverse_complement(rev.ref),
                   alt = hl.reverse_complement(rev.alt))
rev = rev.key_by('context', 'ref', 'alt', 'methylation_level')

In [ ]:
mut = mut.union(rev)

mut.count()

In [ ]:
mut = mut.drop('variant_count', 'downsampling_counts_global', 'possible_variants', 
               'downsamplings_frac_observed', 'downsamplings_mu_snp')

In [ ]:
mut.show(208)

In [ ]:
mut.write('gs://my_bucket/Mutation_rates/gnomad.v4.1.mutation_rate_with_rev_comp_2024-07-17.ht')
# Wrote table with 208 rows in 2 partitions
# to gs://my_bucket/Mutation_rates/gnomad.v4.1.mutation_rate_with_rev_comp_2024-07-17.ht

In [ ]:
# Because I may need it below, what is the minimum mu_snp value?
mut.aggregate(hl.agg.min(mut.mu_snp))

# 9.114551589132187e-10

In [ ]:
#####

In [ ]:
#####
##### OK, now let's annotate all exonic SNVs with corresponding mutation rates
#####

In [ ]:
#####

In [ ]:
# Read table with all GRCh38 SNVs
ht = hl.read_table('gs://gcp-public-data--gnomad/resources/context/grch38_context_vep_annotated.ht')

ht.count() # 8,771,197,662

In [ ]:
ht.n_partitions() # 38029

In [ ]:
ht.show()

In [ ]:
# Define my VEP-based annotations
def add_vep_annotations(mt):
    
    # First, annotate based on parsing VEP output
    mt = mt.annotate(
        gene = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).gene_symbol,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).gene_symbol,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).gene_symbol,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).gene_symbol),
        gene_id = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).gene_id,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).gene_id,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).gene_id,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).gene_id),
        transcript_id = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).transcript_id,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).transcript_id,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).transcript_id,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).transcript_id),
        hgnc_id = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).hgnc_id,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).hgnc_id,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).hgnc_id,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).hgnc_id),
        hgvsc = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).hgvsc,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).hgvsc,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).hgvsc,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).hgvsc),
        hgvsp = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).hgvsp,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).hgvsp,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).hgvsp,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).hgvsp),
        consequence = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).consequence_terms,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).consequence_terms,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).consequence_terms,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).consequence_terms),
        lof = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).lof,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).lof,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).lof,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).lof),
        lof_flags = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).lof_flags,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).lof_flags,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).lof_flags,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).lof_flags),
        polyphen = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).polyphen_prediction,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).polyphen_prediction,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).polyphen_prediction,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).polyphen_prediction),
        sift = hl.coalesce(
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") & (hl.is_defined(x.amino_acids)) ).sift_prediction,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) & (x.biotype == "protein_coding") ).sift_prediction,
            mt.vep.transcript_consequences.find(lambda x: (x.canonical == 1) ).sift_prediction,
            mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(mt.vep.most_severe_consequence)).sift_prediction) )

    # Now, annotate based on the "consequence" annotation
    mt = mt.annotate(
        isPTV = mt.consequence.contains('frameshift_variant') | mt.consequence.contains('stop_gained') |
                mt.consequence.contains('splice_donor_variant') | mt.consequence.contains('splice_acceptor_variant') |
                mt.consequence.contains('transcript_ablation'),
        isMIS = mt.consequence.contains('missense_variant'),
        isSYN = mt.consequence.contains('synonymous_variant') | mt.consequence.contains('stop_retained_variant'),
        isSRG = mt.consequence.contains('splice_region_variant'),
        isSSL = mt.consequence.contains('start_lost') | mt.consequence.contains('stop_lost'),
        isINF = mt.consequence.contains('inframe_insertion') | mt.consequence.contains('inframe_deletion') )
    
    # And variants have an interesting "most severe" consequence (SII = "severe is interesting")
    mt = mt.annotate(
        SII = (mt.vep.most_severe_consequence == 'frameshift_variant') | (mt.vep.most_severe_consequence == 'stop_gained') |
              (mt.vep.most_severe_consequence == 'splice_donor_variant') | (mt.vep.most_severe_consequence ==  'splice_acceptor_variant') |
              (mt.vep.most_severe_consequence == 'transcript_ablation') | (mt.vep.most_severe_consequence == 'missense_variant') |
              (mt.vep.most_severe_consequence == 'synonymous_variant') | (mt.vep.most_severe_consequence == 'stop_retained_variant') |
              (mt.vep.most_severe_consequence == 'splice_region_variant') | (mt.vep.most_severe_consequence == 'start_lost') | 
              (mt.vep.most_severe_consequence == 'stop_lost') | (mt.vep.most_severe_consequence == 'inframe_insertion') | 
              (mt.vep.most_severe_consequence == 'inframe_deletion') )
    
    # Additionally annotate SNV vs indel
    mt = mt.annotate(isSNV = hl.is_snp(mt.alleles[0], mt.alleles[1]),
                     isIndel = hl.is_indel(mt.alleles[0], mt.alleles[1]))
    
    return(mt)

In [ ]:
# Get Broad evaluation regions
evaluation_regions = hl.import_locus_intervals('gs://gcp-public-data--broad-references/hg38/v0/exome_evaluation_regions.v1.interval_list', 
                                            reference_genome = 'GRCh38')

# I made this one in R from the original evaluation regions file;
# it adds 2 bp to the intervals (and merges the ones that then overlap)
evaluation_regions_pm2 = hl.import_locus_intervals('gs://spark_reprocessing/eval_reg_GRCh38_pm2_2020-02-24.txt', 
                                            reference_genome = 'GRCh38')

In [ ]:
# Add VEP annotations
ht = add_vep_annotations(ht)

# Add evaluation region annotations
ht = ht.annotate(eval_reg = hl.is_defined(evaluation_regions[ht.locus]),
                 eval_reg_pm2 = hl.is_defined(evaluation_regions_pm2[ht.locus]))

# Filter based on annotations, keeping coding variants and variants within 2bp of evaluation regions
ht = ht.filter(ht.isPTV | ht.isMIS | ht.isSYN | ht.isSRG | ht.isSSL | ht.isINF | 
               ht.SII | ht.eval_reg_pm2, keep = True)

# Write temp file
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp1.ht')
# Wrote table with 122118570 rows in 38029 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp1.ht

In [ ]:
# Read back
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp1.ht')

ht.count() # 122,118,570

In [ ]:
# Repartition
ht = ht.repartition(5000)
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp2.ht')
# Wrote table with 122118570 rows in 5000 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp2.ht

In [ ]:
# Read back
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp2.ht')

ht.count() # 122,118,570

In [ ]:
# Repartition again
ht = ht.repartition(500)
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp3.ht')
# Wrote table with 122118570 rows in 500 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp3.ht

In [ ]:
# Read back
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp3.ht')

ht.count() # 122,118,570

In [ ]:
ht = ht.annotate(extended_context = ht.context)
ht = ht.annotate(context = ht.context[2:5],
                 ref = ht.alleles[0],
                 alt = ht.alleles[1])

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp4.ht')
# Wrote table with 122118570 rows in 500 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp4.ht

In [ ]:
# Read back
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp4.ht')

ht.count() # 122,118,570

In [ ]:
# All chromosomes here
ht.aggregate(hl.agg.counter(ht.locus.contig))
# 'chr1': 12290559, 'chr2': 9019455, 'chr3': 6977073, 'chr4': 4865523, 'chr5': 5554191, 
# 'chr6': 6087294, 'chr7': 5821947, 'chr8': 4185909, 'chr9': 4899348,  'chr10': 4732386, 
# 'chr11': 7129248, 'chr12': 6390330, 'chr13': 2230638, 'chr14': 3987861, 'chr15': 4287987, 
# 'chr16': 5249175, 'chr17': 6990120, 'chr18': 1959360, 'chr19': 7891197, 'chr20': 2881923, 
# 'chr21': 1253712, 'chr22': 2588835, 'chrX': 4503960, 'chrY': 306537, 'chrM': 34002                              

In [ ]:
ht.show()

In [ ]:
ht.filter(hl.is_defined(ht.methyl_mean)).count()
# 7,262,943

In [ ]:
7262943 / ht.count() # Only about 5.9% of variants have a methyl_mean value

In [ ]:
# Are they all CpG sites?
ht.filter(hl.is_defined(ht.methyl_mean) & ( 
    (ht.context == 'ACG') | (ht.context == 'CCG') | (ht.context == 'GCG') | (ht.context == 'TCG') |
    (ht.context == 'CGT') | (ht.context == 'CGG') | (ht.context == 'CGC') | (ht.context == 'CGA'))).count()
# 7262943 -- OK, this makes sense

In [ ]:
# Can we use the methylation info in the file already?
ht = ht.annotate(methyl_mean = hl.float(ht.methyl_mean))

In [ ]:
# These are clearly not on the 0-15 scale
ht.aggregate(hl.agg.stats(ht.methyl_mean))
# Struct(mean=62.38647712406255, stdev=37.62468633363724, min=0.0, 
# max=99.7460317460317, n=7262943, sum=453109427.32287025)

In [ ]:
# So let's get methylation annotations

In [ ]:
# Here are methylation levels for the autosome...
#
# This is generated in Siwei Chen's whole genome constraint paper:
# https://www.nature.com/articles/s41586-023-06045-0
methyl = hl.import_bed('gs://my_bucket/Mutation_rates/context_prepared_methyl14_weighted_logit.methyl_level.bed',
                       reference_genome = 'GRCh38')

In [ ]:
ht = ht.annotate(methyl_level = hl.int(methyl[ht.locus].target))

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp5.ht')
# Wrote table with 122118570 rows in 500 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp5.ht

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp5.ht')

ht.count() # 122,118,570

In [ ]:
# We have the levels now
ht.aggregate(hl.agg.counter(ht.methyl_level))

In [ ]:
122118570-115106319 # 7012251

In [ ]:
# 96.5% of CpG sites in the dataset get one of these values
7012251/7262943

In [ ]:
# But we didn't get any of chromosome X
ht.filter(hl.is_defined(ht.methyl_level) & (ht.locus.contig == 'chrX'), keep = True).count() # 0

In [ ]:
# So let's get chromosome X

In [ ]:
methyl = hl.import_bed('gs://my_bucket/Mutation_rates/context_prepared_methyl12_weighted_logit_x.methyl_level.bed',
                       reference_genome = 'GRCh38')

In [ ]:
ht = ht.annotate(methyl_level = hl.if_else(hl.is_defined(ht.methyl_level), ht.methyl_level,
                                                         hl.int(methyl[ht.locus].target)))

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp6.ht')
# Wrote table with 122118570 rows in 500 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp6.ht

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp6.ht')

ht.count() # 122,118,570

In [ ]:
# We have more levels now
ht.aggregate(hl.agg.counter(ht.methyl_level))

In [ ]:
122118570-114895665 # 7222905

In [ ]:
# 99.4% of CpG sites in the dataset get one of these values now
7222905/7262943

In [ ]:
# And we do now have some on chrX
ht.filter(hl.is_defined(ht.methyl_level) & (ht.locus.contig == 'chrX'), keep = True).count() # 210654

In [ ]:
7222905-210654 # 7012251 -- what we had before, so all gains were on chrX

In [ ]:
ht = ht.annotate(isCPG = ( ( (ht.context == 'ACG') | (ht.context == 'CCG') | 
                             (ht.context == 'GCG') | (ht.context == 'TCG') ) & (ht.alt == 'T') ) |
                         ( ( (ht.context == 'CGA') | (ht.context == 'CGC') | 
                             (ht.context == 'CGG') | (ht.context == 'CGT') ) & (ht.alt == 'A') ) )

In [ ]:
# Translate this to levels used in the mutation rate file
# (via https://github.com/broadinstitute/gnomad-constraint/blob/c96a574e644d2f2d0f553812168c2efb5f11ce65/gnomad_constraint/utils/constraint.py#L138)
# Note that the github document didn't specifically deal with chrY or chrM, so I have lumped them with chrX here
ht = ht.annotate(methylation_level = hl.case()
                                      .when(hl.is_missing(ht.methyl_level), 0)
                                      .when(ht.isCPG & (
                                            ( ht.locus.in_autosome() & (ht.methyl_level > 5) ) |
                                            (~ht.locus.in_autosome() & (ht.methyl_level > 3) ) ), 2)
                                      .when(ht.isCPG & (ht.methyl_level > 0), 1)
                                      .default(0))

In [ ]:
# Here it is with only three levels
ht.aggregate(hl.agg.counter(ht.methylation_level))

In [ ]:
294889+1492111 # 1787000

In [ ]:
1787000/7222905 # 24.7% of those sites with a value get higher than level 0

In [ ]:
1787000+120331570

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp7.ht')
# Wrote table with 122118570 rows in 500 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp7.ht

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp7.ht')

ht.count() # 122,118,570

In [ ]:
ht = ht.key_by('context', 'ref', 'alt', 'methylation_level')

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp8.ht')
# Wrote table with 122118570 rows in 194 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp8.ht

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp8.ht')

ht.count() # 122,118,570

In [ ]:
# Read back mutation rates
mut = hl.read_table('gs://my_bucket/Mutation_rates/gnomad.v4.1.mutation_rate_with_rev_comp_2024-07-17.ht')

mut.count() # 208

In [ ]:
# Add mutation rates to each variant in sites file
ht = ht.annotate(mu_snp = mut[ht.key].mu_snp)

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp9.ht')
# Wrote table with 122118570 rows in 194 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp9.ht

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_temp9.ht')

ht.count() # 122,118,570

In [ ]:
ht.show()

In [ ]:
ht.filter(hl.is_missing(ht.mu_snp), keep = True).count()

# 6 --> OK

In [ ]:
# There are only 6 non-standard context values, so I'll give those the min value for mu
ht.aggregate(hl.agg.counter(ht.context))

In [ ]:
ht = ht.annotate(mu_snp = hl.if_else(hl.is_defined(ht.mu_snp), ht.mu_snp, 9.114551589132187e-10))

In [ ]:
ht = ht.key_by('locus', 'alleles')

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter.ht')
# Wrote table with 122118570 rows in 194 partitions
# to gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter.ht

In [ ]:
# Check that CPG annotation makes sense
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter.ht')

ht.count() # 122,118,570

In [ ]:
ht.filter(ht.isCPG, keep = True).count() # 2421595

In [ ]:
temp = ht.filter(ht.context == 'CCG', keep = True)

temp.aggregate(hl.agg.counter(temp.alt))

In [ ]:
tempA = temp.filter(temp.alt == 'A', keep = True)
tempG = temp.filter(temp.alt == 'G', keep = True)
tempT = temp.filter(temp.alt == 'T', keep = True) 
# Only the C -> T should be CpG

In [ ]:
a = tempA.filter(tempA.isCPG, keep = True).count()
b = tempA.filter(hl.is_defined(tempA.methyl_mean), keep = True).count()
c = tempA.filter(hl.is_defined(tempA.methyl_level), keep = True).count()
d = tempA.aggregate(hl.agg.counter(tempA.methylation_level))

print(a); print(b); print(c); print(d)
# 0, 395175, 393130, {0: 395271}

In [ ]:
a = tempG.filter(tempG.isCPG, keep = True).count()
b = tempG.filter(hl.is_defined(tempG.methyl_mean), keep = True).count()
c = tempG.filter(hl.is_defined(tempG.methyl_level), keep = True).count()
d = tempG.aggregate(hl.agg.counter(tempG.methylation_level))

print(a); print(b); print(c); print(d)
# 0, 395175, 393130, {0: 395271}

In [ ]:
a = tempT.filter(tempT.isCPG, keep = True).count()
b = tempT.filter(hl.is_defined(tempT.methyl_mean), keep = True).count()
c = tempT.filter(hl.is_defined(tempT.methyl_level), keep = True).count()
d = tempT.aggregate(hl.agg.counter(tempT.methylation_level))

print(a); print(b); print(c); print(d)
# 395271, 395175, 393130, {0: 109386, 1: 49308, 2: 236577}

In [ ]:
ht.filter(hl.is_missing(ht.mu_snp), keep = True).count()

# 0 --> OK

In [ ]:
# Proceed...

In [ ]:
### Annotate with lifted-over MPC values

# Import and count
mpc = hl.read_table('gs://my_bucket/mpc.GRCh38.ht')
mpc.count() # 66,786,545

# Need to make the "alleles" an array
mpc = mpc.annotate(allele_array = [mpc.alleles[0], mpc.alleles[1]])

# Key it by both locus and alleles
mpc = mpc.key_by(mpc.locus_38, mpc.allele_array)

# Annotate
ht = ht.annotate(MPC = mpc[ht.locus, ht.alleles].MPC)

In [ ]:
### Fix chr22 MPC annotation

# Get new chr22 MPC values
mpc_chr22 = hl.read_table('gs://my_bucket/MPC_chr22_liftover/mpc_chr22_GRCh38_2023-09-15.ht')

# Get initial number of rows
a = ht.count()

# Split
ht_chr22 = ht.filter(ht.locus.contig == 'chr22', keep = True)
ht_other = ht.filter(ht.locus.contig != 'chr22', keep = True)

# Annotate
ht_chr22 = ht_chr22.annotate(Old_MPC = ht_chr22.MPC)
ht_chr22 = ht_chr22.annotate(MPC = mpc_chr22[ht_chr22.key].MPC)

# Check
b = ht_chr22.filter(hl.is_defined(ht_chr22.MPC), keep = True).count()
c = ht_chr22.filter(hl.is_defined(ht_chr22.Old_MPC), keep = True).count()
print(b > c) # True

ht_chr22 = ht_chr22.drop('Old_MPC')

# Merge back and check
ht = ht_other.union(ht_chr22)
d = ht.count()
print(a == d) # True

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp1.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp1.ht')

ht.count() # 122,118,570

In [ ]:
### Annotate with gnomAD exome frequencies (gnomad v2.1.1)
# Note the updated path

# Use "non-neuro" allele frequencies 
gnomad_ht = hl.read_table('gs://gcp-public-data--gnomad/release/2.1.1/liftover_grch38/ht/exomes/gnomad.exomes.r2.1.1.sites.liftover_grch38.ht')

# Add frequencies
ht = ht.annotate(     gnomad_non_neuro_AF = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro"])].AF,
                      gnomad_non_neuro_AF_afr = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_afr"])].AF,
                      gnomad_non_neuro_AF_amr = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_amr"])].AF,
                      gnomad_non_neuro_AF_asj = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_asj"])].AF,
                      gnomad_non_neuro_AF_eas = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_eas"])].AF,
                      gnomad_non_neuro_AF_fin = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_fin"])].AF,
                      gnomad_non_neuro_AF_nfe = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_nfe"])].AF,
                      gnomad_non_neuro_AF_oth = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_oth"])].AF,
                      gnomad_non_neuro_AF_sas = 
                         gnomad_ht.index(ht.key).freq[hl.eval(gnomad_ht.freq_index_dict["non_neuro_sas"])].AF
                      )

# Set missing values to zero
ht = ht.annotate(     gnomad_non_neuro_AF = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF), 
                                                       ht.gnomad_non_neuro_AF, 0),
                      gnomad_non_neuro_AF_afr = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_afr), 
                                                       ht.gnomad_non_neuro_AF_afr, 0),
                      gnomad_non_neuro_AF_amr = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_amr), 
                                                       ht.gnomad_non_neuro_AF_amr, 0),
                      gnomad_non_neuro_AF_asj = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_asj), 
                                                       ht.gnomad_non_neuro_AF_asj, 0),
                      gnomad_non_neuro_AF_eas = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_eas), 
                                                       ht.gnomad_non_neuro_AF_eas, 0),
                      gnomad_non_neuro_AF_fin = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_fin), 
                                                       ht.gnomad_non_neuro_AF_fin, 0),
                      gnomad_non_neuro_AF_nfe = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_nfe), 
                                                       ht.gnomad_non_neuro_AF_nfe, 0),
                      gnomad_non_neuro_AF_oth = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_oth), 
                                                       ht.gnomad_non_neuro_AF_oth, 0),
                      gnomad_non_neuro_AF_sas = hl.if_else(hl.is_defined(ht.gnomad_non_neuro_AF_sas), 
                                                       ht.gnomad_non_neuro_AF_sas, 0) )

# Annotate subpop max
ht = ht.annotate(     gnomad_non_neuro_AF_popmax = hl.max(ht.gnomad_non_neuro_AF_afr, ht.gnomad_non_neuro_AF_amr, 
                                                          ht.gnomad_non_neuro_AF_asj, ht.gnomad_non_neuro_AF_eas, 
                                                          ht.gnomad_non_neuro_AF_fin, ht.gnomad_non_neuro_AF_nfe, 
                                                          ht.gnomad_non_neuro_AF_oth, ht.gnomad_non_neuro_AF_sas))


In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp2.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp2.ht')

ht.count() # 122,118,570

In [ ]:
ht = ht.repartition(400)
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp3.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp3.ht')

ht.count() # 122,118,570

In [ ]:
ht = ht.drop('idx', 'a_index', 'was_split', 'old_locus', 'old_alleles', 'polyphen', 'sift', 'ref', 'alt',
             'gnomad_non_neuro_AF_afr', 'gnomad_non_neuro_AF_amr', 'gnomad_non_neuro_AF_asj', 
             'gnomad_non_neuro_AF_eas', 'gnomad_non_neuro_AF_fin', 'gnomad_non_neuro_AF_nfe',
             'gnomad_non_neuro_AF_oth', 'gnomad_non_neuro_AF_sas')

In [ ]:
# Get Alpha Missense (and MisFit_S) values
am38 = hl.read_table('gs://my_bucket/ProteinStability/AlphaMissense_deduplicated_hg38_with_ps_mf_2024-05-21.ht')
am38 = am38.key_by('locus', 'alleles')

# Add values to dataset
ht = ht.annotate(am_pathogenicity = am38[ht.key].am_pathogenicity,
                 MisFit_S = am38[ht.key].MisFit_S)

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp4.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp4.ht')

ht.count() # 122,118,570

In [ ]:
# Get MPC_v2 (from gnomad 2.1.1) values
mpc_v2 = hl.read_table('gs://my_bucket/mpc_gnomad_2.1.1/mpc_grch38_deduped_with_outliers_2024-04-30.ht').key_by('locus', 'alleles')
mpc_v2 = mpc_v2.key_by('locus', 'alleles')

# Add MPC_v2 values to dataset
ht = ht.annotate(MPC_v2 = mpc_v2[ht.key].mpc,
                 MPC_v2_Outlier = mpc_v2[ht.key].mpc_outlier)

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp5.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp5.ht')

ht.count() # 122,118,570

In [ ]:
# Print the proportion of missense sites that were annotated
a = ht.filter(ht.isMIS, keep = True).count()

b = ht.filter(ht.isMIS & hl.is_defined(ht.am_pathogenicity), keep = True).count()
print(b/a) # 95.7%

c = ht.filter(ht.isMIS & hl.is_defined(ht.MisFit_S), keep = True).count()
print(c/a) # 93.8%

d = ht.filter(ht.isMIS & hl.is_defined(ht.MPC_v2), keep = True).count()
print(d/a) # 95.8%

In [ ]:
### Annotating low-complexity regions ###
lcr = hl.import_bed('gs://my_bucket/LCR-hs38.bed', reference_genome = 'GRCh38')
# This file is from https://github.com/lh3/varcmp/blob/master/scripts/LCR-hs38.bed.gz

In [ ]:
# Annotate LCR
ht = ht.annotate(inLCR = hl.is_defined(lcr[ht.locus]))

ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp6.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_temp6.ht')

ht.count() # 122,118,570

In [ ]:
ht.show()

In [ ]:
ht = ht.drop('vep', 'hgnc_id', 'SII', 'isSNV', 'eval_reg')

In [ ]:
# Get "other splice" variants
os_vars = hl.read_table('gs://my_bucket/splice/20240618_GRCh38_OS.ht')

os_vars.count() # 931684

In [ ]:
ht = ht.annotate(isOS = hl.is_defined(os_vars[ht.key]))

In [ ]:
# Write fully annotated sites file
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot.ht')

In [ ]:
# Read back annotated sites file
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot.ht')

ht.count() # 122,118,570

In [ ]:
ht.show()

In [ ]:
# Remove LCR sites
ht = ht.filter(ht.inLCR == False, keep = True)

ht.count() # 121,662,717

In [ ]:
121662717/122118570 # Keep 99.6% of sites

In [ ]:
# Remove sites with popmax above 0.1%
ht = ht.filter(ht.gnomad_non_neuro_AF_popmax <= 0.001, keep = True)

ht.count() # 121,107,483

In [ ]:
121107483 / 121662717 # 99.5% of non-LCR sites (99.2% of total)

In [ ]:
# Fix double counting
ht = ht.annotate(isMIS = hl.if_else(ht.isPTV, False, ht.isMIS))
ht = ht.annotate(isSYN = hl.if_else(ht.isPTV, False, ht.isSYN))
ht = ht.annotate(isSYN = hl.if_else(ht.isMIS, False, ht.isSYN))

In [ ]:
# Let's annotate exactly the categories we want
ht = ht.annotate(isPTV_for_dn = (ht.isPTV | ht.isOS),
                 isMis2_for_dn = (ht.isMIS & (~ht.isOS) & (ht.MPC_v2 >= 2) & (ht.am_pathogenicity >= 0.97)),
                 isMis1_for_dn = (ht.isMIS & (~ht.isOS) & (
                     ((ht.MPC_v2 >= 2) & ((ht.am_pathogenicity < 0.97) | hl.is_missing(ht.am_pathogenicity))) | 
                     ((ht.am_pathogenicity >= 0.97) & ((ht.MPC_v2 < 2) | hl.is_missing(ht.MPC_v2))) ) ),
                 isMis0_for_dn = (ht.isMIS & (~ht.isOS) &
                     ((ht.am_pathogenicity < 0.97) | hl.is_missing(ht.am_pathogenicity)) & 
                     ((ht.MPC_v2 < 2) | hl.is_missing(ht.MPC_v2)) ),
                 isSyn_for_dn = ht.isSYN & (~ht.isOS),
# Let's include older categories, too
                 isMisB_for_dn = (ht.isMIS & (~ht.isOS) & (ht.MPC >= 2)),
                 isMisA_for_dn = (ht.isMIS & (~ht.isOS) & (ht.MPC >= 1) & (ht.MPC < 2)),
                 isMisOther_for_dn = (ht.isMIS & (~ht.isOS) & ((ht.MPC < 1) | (hl.is_missing(ht.MPC))) ))

In [ ]:
# Write annotated and prepped sites file
ht.write('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_prep.ht')

In [ ]:
# Read back annotated and prepped sites file
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/grch38_context_vep_filter_annot_prep.ht')

ht.count() # 121,107,483

In [ ]:
ht.show()

In [ ]:
# How many sites are in the autosome or PAR?
ht.filter(ht.locus.in_autosome_or_par(), keep = True).count() # 116384981

In [ ]:
# How many sites are chrX non-PAR?
ht.filter(ht.locus.in_x_nonpar(), keep = True).count() # 4384120

In [ ]:
# How many sites are chrY non-PAR?
ht.filter(ht.locus.in_y_nonpar(), keep = True).count() # 304380

In [ ]:
# How many sites are chrM?
ht.filter(ht.locus.in_mito(), keep = True).count() # 304380

In [ ]:
# This is all the contigs
116384981+4384120+304380+34002 == 121107483 # True

In [ ]:
# What do the gnomad coverage stats look like across contigs?

In [ ]:
ht2 = ht.filter(ht.locus.in_autosome_or_par(), keep = True)

ht2.aggregate(hl.agg.stats(ht2.coverage_mean)) # Mean 30.95

In [ ]:
ht2 = ht.filter(ht.locus.in_x_nonpar(), keep = True)

ht2.aggregate(hl.agg.stats(ht2.coverage_mean)) # Mean 22.98

In [ ]:
ht2 = ht.filter(ht.locus.in_y_nonpar(), keep = True)

ht2.aggregate(hl.agg.stats(ht2.coverage_mean)) # Mean 8.45

In [ ]:
ht2 = ht.filter(ht.locus.in_mito(), keep = True)

ht2.aggregate(hl.agg.stats(ht2.coverage_mean)) # Mean NaN --> I'll lose all the chrM loci if filter on coverage

In [ ]:
# What proportion of sites pass a threshold of 10 (or 7)?

In [ ]:
ht2 = ht.filter(ht.locus.in_autosome_or_par(), keep = True)

ht2.filter(ht2.coverage_mean >= 10, keep = True).count() / ht2.count() # 98.7%

In [ ]:
ht2 = ht.filter(ht.locus.in_x_nonpar(), keep = True)

ht2.filter(ht2.coverage_mean >= 10, keep = True).count() / ht2.count() # 95.2%

In [ ]:
ht2 = ht.filter(ht.locus.in_x_nonpar(), keep = True)

ht2.filter(ht2.coverage_mean >= 7, keep = True).count() / ht2.count() # 95.5%

In [ ]:
ht2 = ht.filter(ht.locus.in_x_nonpar(), keep = True)

ht2.filter(ht2.coverage_mean >= 1, keep = True).count() / ht2.count() # 99.0%

In [ ]:
ht2 = ht.filter(ht.locus.in_y_nonpar(), keep = True)

ht2.filter(ht2.coverage_mean >= 10, keep = True).count() / ht2.count() # 50.8%

In [ ]:
ht2 = ht.filter(ht.locus.in_y_nonpar(), keep = True)

ht2.filter(ht2.coverage_mean >= 7, keep = True).count() / ht2.count() # 52.5%

In [ ]:
ht2 = ht.filter(ht.locus.in_y_nonpar(), keep = True)

ht2.filter(ht2.coverage_mean >= 1, keep = True).count() / ht2.count() # 79.8%

In [ ]:
ht2 = ht.filter(ht.locus.in_mito(), keep = True)

ht2.filter(ht2.coverage_mean >= 1, keep = True).count() / ht2.count() # 0

In [ ]:
# Let's do a single read for non-PAR X, non-PAR Y, and mito

In [ ]:
# Filter on coverage so that sites that are poorly covered in gnomad to not play into our expectations
ht = ht.filter( ( ( ht.locus.in_autosome_or_par() ) & (ht.coverage_mean >= 10) ) |
                ( (~ht.locus.in_autosome_or_par() ) & (ht.coverage_mean >= 1 ) ), keep = True)

ht.count() # 119,419,563 (98.6% of 121,107,483)

In [ ]:
# Now continue... sum mutation rate values by gene and variant type

In [ ]:
ht = ht.drop('vep_config', 'grange', 'vep_help')

In [ ]:
# Do a whole bunch of aggregations
var_list = ht.filter(ht.isPTV_for_dn, keep = True)

PTV_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_PTV = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isMis2_for_dn, keep = True)

Mis2_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_Mis2 = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isMis1_for_dn, keep = True)

Mis1_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_Mis1 = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isMis0_for_dn, keep = True)

Mis0_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_Mis0 = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isSyn_for_dn, keep = True)

Syn_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_Syn = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isMisB_for_dn, keep = True)

MisB_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_MisB = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isMisA_for_dn, keep = True)

MisA_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_MisA = hl.agg.sum(var_list.mu_snp) ))

var_list = ht.filter(ht.isMisOther_for_dn, keep = True)

MisOther_table = (var_list.group_by(var_list.gene_id).aggregate(
    gene = hl.agg.take(var_list.gene, 1)[0],
    chrom = hl.agg.take(var_list.locus.contig, 1)[0],
    mu_snp_MisOther = hl.agg.sum(var_list.mu_snp) ))

In [ ]:
# Do a whole bunch of joins

# Join PTV & Mis2
all_table = PTV_table.join(Mis2_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')

# Add Mis1
all_table = all_table.join(Mis1_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')

# Add Mis0
all_table = all_table.join(Mis0_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')

# Add Syn
all_table = all_table.join(Syn_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')

# Add MisB
all_table = all_table.join(MisB_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')

# Add MisA
all_table = all_table.join(MisA_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')

# Add MisOther
all_table = all_table.join(MisOther_table, how = 'outer')

all_table = all_table.annotate(gene = hl.if_else(hl.is_missing(all_table.gene), all_table.gene_1, all_table.gene),
                               chrom = hl.if_else(hl.is_missing(all_table.chrom), all_table.chrom_1, all_table.chrom))
all_table = all_table.drop('gene_1', 'chrom_1')


In [ ]:
all_table.write('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_temp1_2024-07-24.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_temp1_2024-07-24.ht')

ht.count() # 29,380

In [ ]:
ht = ht.filter(hl.is_defined(ht.gene_id), keep = True)

In [ ]:
ht = ht.annotate(mu_snp_PTV = hl.if_else(hl.is_missing(ht.mu_snp_PTV), 0, ht.mu_snp_PTV),
                 mu_snp_Mis2 = hl.if_else(hl.is_missing(ht.mu_snp_Mis2), 0, ht.mu_snp_Mis2),
                 mu_snp_Mis1 = hl.if_else(hl.is_missing(ht.mu_snp_Mis1), 0, ht.mu_snp_Mis1),
                 mu_snp_Mis0 = hl.if_else(hl.is_missing(ht.mu_snp_Mis0), 0, ht.mu_snp_Mis0),
                 mu_snp_Syn = hl.if_else(hl.is_missing(ht.mu_snp_Syn), 0, ht.mu_snp_Syn),
                 mu_snp_MisB = hl.if_else(hl.is_missing(ht.mu_snp_MisB), 0, ht.mu_snp_MisB),
                 mu_snp_MisA = hl.if_else(hl.is_missing(ht.mu_snp_MisA), 0, ht.mu_snp_MisA),
                 mu_snp_MisOther = hl.if_else(hl.is_missing(ht.mu_snp_MisOther), 0, ht.mu_snp_MisOther) )

In [ ]:
ht.write('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_temp2_2024-07-24.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_temp2_2024-07-24.ht')

ht.count() # 29,379

In [ ]:
ht = ht.repartition(5)

In [ ]:
# Write final Hail table
ht.write('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_2024-07-24.ht')

In [ ]:
ht = hl.read_table('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_2024-07-24.ht')

ht.count() # 29,379

In [ ]:
ht.show()

In [ ]:
ht.export('gs://my_bucket/Mutation_rates/Imported/ASD_mutation_rates_draft_2024-07-24.txt')
# Note that further formatting occurred in R